In [1]:
#*************************
import pickle
import pandas as pd
import numpy as np
import sqlalchemy
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display
import sys

In [2]:
#*************************
url = "dbbikes.cvvkn6jkfbdf.eu-west-1.rds.amazonaws.com"
port = "3306"
db = "dbbikes"
user = "SEGroup25"
password = "SEGroup25?"



In [3]:
#*************************
engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)


In [4]:
#*************************
availability_sql = pd.read_sql("select * from dbbikes.availability", engine)

2023-04-12 21:15:34,894 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-12 21:15:34,897 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-12 21:15:34,927 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-12 21:15:34,929 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-12 21:15:34,950 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-12 21:15:34,951 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-12 21:15:35,009 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-12 21:15:35,012 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ('dbbikes', 'select * from dbbikes.availability')
2023-04-12 21:15:35,059 INFO sqlalchemy.engine.Engine select * from dbbikes.availability
2023-04-12 21:15:35,060 INFO sqlalchemy.engine.Engine [raw sql] ()


In [5]:
#*************************
weather_sql = pd.read_sql("select * from dbbikes.weather", engine)

2023-04-12 21:15:38,899 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-12 21:15:38,900 INFO sqlalchemy.engine.Engine [cached since 3.892s ago] ('dbbikes', 'select * from dbbikes.weather')
2023-04-12 21:15:38,935 INFO sqlalchemy.engine.Engine select * from dbbikes.weather
2023-04-12 21:15:38,936 INFO sqlalchemy.engine.Engine [raw sql] ()


In [6]:
#*************************
weather_sql.shape[0]

701

In [7]:
#*************************
availability_sql.shape

(457595, 4)

In [8]:
#*************************
availability_sql.head(5)

,available_bike_stands,available_bikes,last_update,number
0,23,7,2147483647,42
1,14,6,2147483647,30
2,21,12,2147483647,54
3,21,14,2147483647,108
4,14,16,2147483647,20


In [9]:
#*************************
# group by "category" and select first 300 rows of each group
df_trimmed = availability_sql.groupby('number').tail(weather_sql.shape[0])

# reset index of trimmed dataframe
df_trimmed = df_trimmed.reset_index(drop=True)

In [10]:
#*************************
df_trimmed.shape # output: 3

(80067, 4)

In [11]:
#*************************
df_trimmed['index'] = df_trimmed.groupby('number').cumcount()
merged_df = df_trimmed.join(weather_sql,on='index', how='left')
merged_df.head()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
0,0,1,2147483647,507,0,2023-03-30 13:53:20,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
1,0,1,2147483647,507,1,2023-03-30 13:55:03,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
2,0,1,2147483647,507,2,2023-03-30 14:11:04,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
3,0,1,2147483647,507,3,2023-03-30 14:11:26,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
4,0,1,2147483647,507,4,2023-03-30 14:15:02,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13


In [12]:
#*************************
label_encoder = preprocessing.LabelEncoder()
merged_df['weather_main'] = label_encoder.fit_transform(merged_df['weather_main'])

In [13]:
#*************************
merged_df.tail()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
80062,7,12,2147483647,39,700,2023-04-12 20:15:02,-6.2672,53.344,1,6.86,81.0,10.8,280.0,2023-04-12 19:18:16,2023-04-12 05:33:12
80063,25,15,2147483647,83,700,2023-04-12 20:15:02,-6.2672,53.344,1,6.86,81.0,10.8,280.0,2023-04-12 19:18:16,2023-04-12 05:33:12
80064,19,21,2147483647,92,700,2023-04-12 20:15:02,-6.2672,53.344,1,6.86,81.0,10.8,280.0,2023-04-12 19:18:16,2023-04-12 05:33:12
80065,17,13,2147483647,21,700,2023-04-12 20:15:02,-6.2672,53.344,1,6.86,81.0,10.8,280.0,2023-04-12 19:18:16,2023-04-12 05:33:12
80066,13,17,2147483647,88,700,2023-04-12 20:15:02,-6.2672,53.344,1,6.86,81.0,10.8,280.0,2023-04-12 19:18:16,2023-04-12 05:33:12


In [14]:
#*************************
merged_df['month'] = merged_df['time'].dt.month
merged_df['dayofweek'] = merged_df['time'].dt.dayofweek
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute

In [15]:
print(merged_df.isnull().sum().sum())

0


In [16]:
#*************************
merged_df.dtypes

available_bike_stands             int64
available_bikes                   int64
last_update                       int64
number                            int64
index                             int64
time                     datetime64[ns]
cord_long                       float64
cord_lat                        float64
weather_main                      int64
main_temp                       float64
main_humidity                   float64
wind_speed                      float64
wind_degree                     float64
sys_sunset               datetime64[ns]
sys_sunrise              datetime64[ns]
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [17]:
merged_df.shape

(80067, 19)

In [18]:
#*************************
X = merged_df[['number','month','hour','minute','weather_main','main_temp','main_humidity','wind_speed']]
Y = merged_df.available_bikes
X_train, X_test,y_train,y_test = train_test_split(X, Y, test_size=0.3,random_state=0)
regressor = RandomForestRegressor(n_estimators=merged_df['number'].nunique())

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Write to a pickle file
# with open('monday_station.pkl', 'wb') as handle:
#     pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)

print('Test Set Predictions for Station: ')
print(df)
print()

#Accuracy score
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Test Set Predictions for Station: 
       Actual  Predicted
76216       0   0.000000
79460      14  13.982609
8893       35  37.139130
76566      15  15.834783
13323       1  12.269565
...       ...        ...
9395        5  15.173913
36580       9   9.182609
78941       9   9.069565
33783      22  20.913043
74332      16  15.973913

[24021 rows x 2 columns]

Mean Absolute Error: 2.133205184593915
Mean Squared Error: 15.04317809503326
Root Mean Squared Error: 3.8785536086321226
The accuracy score :  0.7952559369670316
